In [8]:
import collections.abc
collections.Mapping = collections.abc.Mapping
collections.MutableMapping = collections.abc.MutableMapping
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import numpy as np
import tensorflow as tf
import json
import random
import warnings

from spellchecker import SpellChecker
import pandas as pd
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="Model was constructed with shape (None, None, 47, 121) for input KerasTensor.*")





In [9]:
import streamlit as st




In [10]:

import tensorflow as tf
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = tf.keras.models.load_model("model2")
confidence_threshold = 0.5

def is_unanswerable_question(question):
    inputs = tokenizer.encode_plus(question, return_tensors="tf", add_special_tokens=True)
    start_logits, end_logits = model(inputs['input_ids'])

    start_probabilities = tf.nn.softmax(start_logits, axis=1)
    end_probabilities = tf.nn.softmax(end_logits, axis=1)

    answerable_confidence = tf.reduce_max(start_probabilities).numpy()
    return answerable_confidence < confidence_threshold


Model name 'distilbert-base-cased' was not found in model name list (distilbert-base-uncased, distilbert-base-uncased-distilled-squad). We assumed 'distilbert-base-cased' was a path or url but couldn't find tokenizer filesat this path or url.


In [11]:

def tokenize(question):
    return word_tokenize(question)
    
def stem_word(word, stemmer = PorterStemmer()):
    return stemmer.stem(word)

def data():
    with open("tags.json", "r") as file:
        tags = json.load(file)
    with open("all_words.json", "r") as file:
        all_words = json.load(file)
    return tags,all_words


def bag_of_words(vocab,tok_sentence):
    bag = np.zeros(len(vocab), dtype=np.float32)
    for ind, word in enumerate(vocab):
        if word in tok_sentence:
            bag[ind] = 1.0
    return bag

def predict(input, tags):
    model = tf.keras.models.load_model("model2")
    prediction = tags[np.argmax(model.predict(input))]
    max_prob = max(model.predict(input)[0])
    return prediction, max_prob






def question_understood(question):
    pass

def get_answer(prediction):
    with open("intents.json", "r") as data:
        intents = json.load(data)
        for tg in intents["intents"]:
            if tg['tag'] == prediction:
                choice = random.choice(tg["responses"])
                break
    return choice
def auto_correct_sentence(sentence):
    corrected_tokens = [spell.correction(token) for token in sentence]
    return corrected_tokens


def process_question(question, vocab):
    tok_ques = tokenize(question)
    stemmed_words = [stem_word(s) for s in tok_ques if s not in [',', '.', '!', '?', ',']]
    cor_ques = auto_correct_sentence(stemmed_words)
    print(cor_ques)
    word_bag = np.array([1.0 if word in cor_ques else 0.0 for word in vocab], dtype=np.float32)
    word_bag = np.expand_dims(word_bag, axis=0)
    return word_bag





In [13]:
tags, vocab = data()
chat = True
spell = SpellChecker()
spell.word_frequency.load_words(vocab)


user = st.chat_message("user")
bot = st.chat_message("assistant")
print("Welcome to arc solution chatbot!")
while(chat):

    prompt = st.chat_input("Enter something")
    if prompt:
        user.write("User:\t", prompt)
        prd, max_prob = predict(process_question(prompt,vocab),tags)
        answer = get_answer(prd)
        bot.write("Answer:\t", answer)
        
        # prd, max_prob = predict(process_question(question,vocab),tags)
    if (question == "end"):
        print("Thanks for using this bot!")
        chat = False
        break
    # print("User:\t", question)
    


    
    # print("Answer:\t", answer)


Welcome to arc solution chatbot!
Thanks for using this bot!
